In [ ]:
import importlib
import business
importlib.reload(business)

In [17]:
# 第一轮规则审核代码 
import pandas as pd
import os
from datetime import datetime
import re
from typing import Tuple, Dict, List
from business import validate_business
from tqdm import tqdm

def first_round_rule_audit(input_file: str) -> pd.DataFrame:
    """
    执行第一轮规则审核并返回完整的审核结果DataFrame
    
    Args:
        input_file: 输入Excel文件路径
    
    Returns:
        df: 包含所有短信审核结果的DataFrame
    """
    # 读取Excel文件
    print(f"读取文件: {input_file}")
    df = pd.read_excel(input_file)
    print(f"成功读取文件，共 {len(df)} 条记录")
    
    # 创建结果列
    df['总体操作类型'] = None
    df['业务操作类型'] = None
    
    # 审核计数器
    pass_count = 0
    fail_count = 0
    error_count = 0
    
    print("=============== 开始执行第一轮：规则审核==============")
    
    # 对每条短信执行规则审核
    for index, row in tqdm(df.iterrows(), total=len(df), desc="规则审核进度", ncols=100):
        
            
        try:
            # 调用业务规则审核
            business_passed, business_reason = validate_business(
                row['产品类型'],
                row['短信内容'], 
                row['短信签名'],
                row.get('账户类型')
            )
            
            # 更新结果
            if business_passed:
                df.loc[index, '总体操作类型'] = '放行'
                pass_count += 1
            else:
                df.loc[index, '总体操作类型'] = '失败'
                fail_count += 1
                
            df.loc[index, '业务操作类型'] = business_reason
            
        except Exception as e:
            print(f"处理行 {index} 时出错: {str(e)}")
            df.loc[index, '总体操作类型'] = '处理错误'
            df.loc[index, '业务操作类型'] = f"处理错误: {str(e)}"
            error_count += 1
    
    # 输出统计信息
    total = len(df)
    print(f"\n规则审核完成:")
    print(f"- 总记录数: {total}")
    print(f"- 通过数量: {pass_count} ({pass_count/total*100:.2f}%)")
    print(f"- 失败数量: {fail_count} ({fail_count/total*100:.2f}%)")
    if error_count > 0:
        print(f"- 错误数量: {error_count} ({error_count/total*100:.2f}%)")
    
    #可选：保存结果到Excel文件
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"123规则审核_{timestamp}.xlsx"
    df.to_excel(output_file, index=False)
    print(f"完整审核结果已保存至: {output_file}")
    
    
    # 统计不同情况的数量
    code_pass_human_fail = len(df[(df['总体操作类型'] == '放行') & (df['操作类型'] == '失败')])
    code_fail_human_pass = len(df[(df['总体操作类型'] == '失败') & (df['操作类型'] == '放行')])
    matched = len(df[df['总体操作类型'] == df['操作类型']])
    match_rate = (matched / total) * 100
    
    # 打印结果
    print('\n审核结果统计:')
    print(f'总样本数: {total}')
    print(f'代码放行但人工失败数量: {code_pass_human_fail}')
    print(f'代码失败但人工放行数量: {code_fail_human_pass}')
    print(f'匹配数量: {matched}')
    print(f'匹配率: {match_rate:.2f}%')


    return df

# 使用示例 
df_with_results = first_round_rule_audit("3月审核记录.xlsx")



读取文件: 3月审核记录.xlsx
成功读取文件，共 8133 条记录
=============== 开始执行第一轮：规则审核==============


规则审核进度: 100%|████████████████████████████████████████████| 8133/8133 [00:19<00:00, 418.19it/s]



规则审核完成:
- 总记录数: 8133
- 通过数量: 556 (6.84%)
- 失败数量: 7577 (93.16%)
完整审核结果已保存至: 123规则审核_20250415_134200.xlsx

审核结果统计:
总样本数: 8133
代码放行但人工失败数量: 30
代码失败但人工放行数量: 6412
匹配数量: 1691
匹配率: 20.79%


In [122]:
import importlib
import business
import ai_check 
importlib.reload(business)
importlib.reload(ai_check )

<module 'ai_check' from '/Users/a92461/Desktop/硕汉/短信审核/ai_check.py'>

In [156]:
import psutil
import os

def show_memory_usage():
    process = psutil.Process(os.getpid())
    print(f"当前内存使用: {process.memory_info().rss / 1024 / 1024:.2f} MB")

In [13]:

import time
from business import validate_business
from typing import Tuple, Dict
import pandas as pd
import os
from datetime import datetime
import re
from collections import Counter
from ai_check import AIAuditor
import json

# 2. 准备AI审核数据
def prepare_ai_data(sample_df):
    """准备AI审核数据"""
    ai_audit_list = []
    for _, row in sample_df.iterrows():
        try:
            score = 100.0  # 默认分数
            try:
                score_match = re.search(r'总分: (\d+\.?\d*)', row['业务操作类型'])
                if score_match:
                    score = float(score_match.group(1))
                elif "直接通过" in row['业务操作类型']:
                    score = 100.0  # 直接通过给高分
            except Exception as e:
                print(f"提取分数时出错: {str(e)}")
            
            ai_audit_list.append({
                "signature": row['短信签名'],
                "content": row['短信内容'],
                "business_type": row['产品类型'],
                "rule_score": score,
                "rule_reason": row['业务操作类型']
            })
        except Exception as e:
            print(f"准备AI审核数据时出错: {str(e)}")
    return ai_audit_list


# 4. 测试小批量审核
def test_batch_audit(df):
    """测试小批量审核"""    
    # 然后在下一个单元格中可以随机抽取部分放行的短信进行AI二次审核
    sample_size = 10  # 设置样本大小
    passed_df = df[df['总体操作类型'] == '放行']
    sample_df = passed_df
    sample_df = sample_df.sample(min(sample_size, len(passed_df)))
    print(f"已随机抽取 {len(sample_df)} 条通过规则审核的短信用于AI二次审核")
    print("============= 开始第二轮：AI审核 =============")
    # 准备AI审核数据
    ai_audit_list = prepare_ai_data(sample_df)

    # 批量审核
    auditor = AIAuditor()
    start_time = time.time()
    results = auditor.batch_audit(ai_audit_list)
    elapsed = time.time() - start_time
    
    # 统计结果
    pass_count = sum(1 for result in results if result['passed'])
    reject_count = len(results) - pass_count
    
    print(f"审核完成: 通过 {pass_count} 条, 驳回 {reject_count} 条")
    print(f"总耗时: {elapsed:.2f}秒, 平均每条: {elapsed/len(results):.2f}秒")
    
    # 显示详细结果
    print("\n详细结果:")
    for i, result in enumerate(results):
        sms = result['sms']
        print(f"\n短信 {i+1}:")
        print(f"签名: {sms['signature']}")
        print(f"内容: {sms['content'][:50]}..." if len(sms['content']) > 50 else f"内容: {sms['content']}")
        print(f"业务类型: {sms['business_type']}")
        print(f"审核结果: {'通过' if result['passed'] else '驳回'}")
        if not result['passed'] and 'reasons' in result['details']:
            print(f"驳回原因: {', '.join(result['details']['reasons'])}")
    
    return results

# 运行测试
# test_single_audit()
test_batch_audit(df_with_results)

2025-04-10 11:55:05,632 - INFO - 开始批量审核 10 条短信
2025-04-10 11:55:05,633 - INFO - 正在处理: 1/10


已随机抽取 10 条通过规则审核的短信用于AI二次审核
============= 开始第二轮：AI审核 =============


2025-04-10 11:55:15,511 - INFO - 正在处理: 2/10
2025-04-10 11:55:21,039 - INFO - 正在处理: 3/10
2025-04-10 11:55:31,165 - INFO - 正在处理: 4/10
2025-04-10 11:55:39,220 - INFO - 正在处理: 5/10
2025-04-10 11:55:46,163 - INFO - 正在处理: 6/10
2025-04-10 11:55:57,428 - INFO - 正在处理: 7/10
2025-04-10 11:56:05,323 - INFO - 正在处理: 8/10
2025-04-10 11:56:11,324 - INFO - 正在处理: 9/10
2025-04-10 11:56:19,937 - INFO - 正在处理: 10/10
2025-04-10 11:56:28,365 - INFO - 批量审核完成，共处理 10 条短信


审核完成: 通过 10 条, 驳回 0 条
总耗时: 82.74秒, 平均每条: 8.27秒

详细结果:

短信 1:
签名: 康喜足道
内容: 【康喜足道】VIP服务升级：每周三会员折上折；市区东区专车接送(12点～0点)；自助餐、四川特色小吃...
业务类型: 会销-普通
审核结果: 通过

短信 2:
签名: GALENIC科兰黎旗舰店
内容: 【GALENIC科兰黎旗舰店】亲亲，逛到喜欢的商品可以买回家试试哦！ https://s.tb.cn...
业务类型: 会销-普通
审核结果: 通过

短信 3:
签名: 薇诺娜官方旗舰店
内容: 【薇诺娜官方旗舰店】防晒会员专享❤清透防晒乳买50g到手80g！报名挑战计划享更多好礼 s.tb.c...
业务类型: 会销-普通
审核结果: 通过

短信 4:
签名: 天猫超市
内容: 【天猫超市】碧芭宝贝早春焕新！大鱼加量65券已到账！到手291元/箱，快来囤货唷~ s.tb.cn/...
业务类型: 会销-普通
审核结果: 通过

短信 5:
签名: 团子家的果园
内容: 【团子家的果园】全网独家上架：博洋酥蜜甜瓜。顶级酥脆香甜！比传统羊角蜜好吃N倍！赠您25元老客券 h...
业务类型: 会销-普通
审核结果: 通过

短信 6:
签名: 滴滴出行
内容: 【滴滴出行】5元立减券已到账！还可免费领6折打车券！点击进入APP查看 https://n.didi...
业务类型: 会销-普通
审核结果: 通过

短信 7:
签名: 永兴国标加油站
内容: 【永兴国标加油站】亲爱的客户您好！我站将于3月28日-4月14日进行升级改造，期间暂停营业，请您做好...
业务类型: 行业-通知
审核结果: 通过

短信 8:
签名: swisse斯维诗海外旗舰店
内容: 【swisse斯维诗海外旗舰店】38节限时加码！K2柠檬酸钙会员领券拍1低至109/瓶 s.tb.c...
业务类型: 会销-普通
审核结果: 通过

短信 9:
签名: 重庆市住房城乡建委
内容: 【重庆市住房城乡建委】住户您好，房管中心温馨提示您，请按时缴纳公租房租金，如已缴纳请忽略。
业务类型: 行业-通知
审核结果: 通过

短信 10:
签名: 中共铜陵市委组织部
内容: 【中共铜陵市委组织部】考生您好，铜陵市“名校优才”招引工作已经开始，报名时间3月

[{'sms': {'signature': '康喜足道',
   'content': '【康喜足道】VIP服务升级：每周三会员折上折；市区东区专车接送(12点～0点)；自助餐、四川特色小吃。详询66555077拒收请回复R',
   'business_type': '会销-普通',
   'rule_score': 65.62,
   'rule_reason': '审核通过 (原因: 基础分: 120, 总分: 65.62)'},
  'passed': True,
  'details': {'should_pass': True,
   'reasons': ['短信内容为正常的商业促销信息，未发现违法违规内容',
    '未涉及政治敏感话题',
    '未涉及色情内容',
    '未发现诈骗嫌疑',
    '未涉及赌博内容',
    '短信中包含拒收选项，符合商业短信规范']}},
 {'sms': {'signature': 'GALENIC科兰黎旗舰店',
   'content': '【GALENIC科兰黎旗舰店】亲亲，逛到喜欢的商品可以买回家试试哦！ https://s.tb.cn/y6.ZOhMJn 拒收请回复R',
   'business_type': '会销-普通',
   'rule_score': 65.62,
   'rule_reason': '审核通过 (原因: 基础分: 120, 总分: 65.62)'},
  'passed': True,
  'details': {'should_pass': True,
   'reasons': ['短信内容不包含政治敏感信息',
    '短信内容不涉及黄色信息',
    '短信内容不涉及诈骗信息',
    '短信内容不涉及赌博信息',
    '短信内容为正常的电商促销信息，符合行业规范']}},
 {'sms': {'signature': '薇诺娜官方旗舰店',
   'content': '【薇诺娜官方旗舰店】防晒会员专享❤清透防晒乳买50g到手80g！报名挑战计划享更多好礼 s.tb.cn/y6.ZlVHsF 拒收请回复R',
   'business_type': '会销-普通',
   'rule_score': 56.84,
   'rule_rea

In [16]:
# 3. 测试单条短信审核
import time
from business import validate_business
from typing import Tuple, Dict
import pandas as pd
import os
from datetime import datetime
import re
from collections import Counter
from ai_check import AIAuditor
import json

def test_single_audit():
    """测试单条短信审核"""
    # 测试一条短信
    test_sms = {
        "signature": " 饿了么 ",
        "content": "【饿了么】尊敬的超级吃货卡用户，我们将在2025-04-02通过支付宝扣除你对应下一周期的卡费，扣款次日即可领取4个红包，扣除的卡费及卡种类请见https://to.ele.me/WPT1ZT9 ",
        "business_type": "行业-物流",
        "account_type": "直客"
    }
    print("=============== 短信信息 =============")
    print(f"签名: {test_sms['signature']}")
    print(f"内容: {test_sms['content']}")
    print(f"业务类型: {test_sms['business_type']}")
    if test_sms.get("account_type"):
        print(f"账户类型: {test_sms['account_type']}")
    print("\n")
    print("=============== 第一轮：规则审核 =============")
    # 执行规则审核
    business_passed, business_reason = validate_business(
        test_sms["business_type"],
        test_sms["content"],
        test_sms["signature"],
        test_sms.get("account_type")
    )   
    # 提取规则审核分数
    score = 100.0  # 默认分数
    try:
        score_match = re.search(r'总分: (-?\d+\.?\d*)', business_reason)
        if score_match:
            score = float(score_match.group(1))
        elif "直接通过" in business_reason:
            score = 100.0
    except Exception as e:
        print(f"提取分数时出错: {str(e)}")
    
    print(f"规则审核结果: {'通过' if business_passed else '失败'}")
    print(f"规则审核原因: {business_reason}")
    print(f"规则审核分数: {score}")
# 运行测试
test_single_audit()

=============== 短信信息 =============
签名:  饿了么 
内容: 【饿了么】尊敬的超级吃货卡用户，我们将在2025-04-02通过支付宝扣除你对应下一周期的卡费，扣款次日即可领取4个红包，扣除的卡费及卡种类请见https://to.ele.me/WPT1ZT9 
业务类型: 行业-物流
账户类型: 直客


=============== 第一轮：规则审核 =============
规则审核结果: 失败
规则审核原因: 审核不通过 (基础分: 100, 总分: -45, 扣分明细:特定关键词签名加分: +20 (匹配关键词: 饿了么), 链接扣分: -50 (链接数量: 1), 固定电话扣分: -40 (检测到: 202504), 
营销关键词扣分: -30 (匹配词: 红包, 领取, 匹配数量: 2), 积分营销扣分: -15 (匹配词: 领取, 匹配数量: 1), 行业催支付关键词扣分: 0 (匹配词: 支付, 匹配数量: 1), 缺少物流关键词扣分: -30)
规则审核分数: -45.0


In [15]:
import importlib
import business
import ai_check 
importlib.reload(business)
importlib.reload(ai_check )

<module 'ai_check' from '/Users/a92461/Desktop/硕汉/短信审核/ai_check.py'>

In [ ]:
    # # 如果规则审核通过，进行AI审核
    # if business_passed:
    #     print("\n=============== 第二轮：AI审核 =============")
    #     auditor = AIAuditor()
    #     start_time = time.time()
    #     passed, details = auditor.audit_sms(
    #         test_sms["signature"],
    #         test_sms["content"],
    #         test_sms["business_type"],
    #         score,
    #         business_reason
    #     )
    #     elapsed = time.time() - start_time
        
    #     print(f"AI审核结果: {'通过' if passed else '驳回'}")
    #     print(f"AI审核详情: {json.dumps(details, ensure_ascii=False, indent=2)}")
    #     print(f"AI审核耗时: {elapsed:.2f}秒")
    # else:
    #     print("\n规则审核未通过，不进行AI审核")

    # # 输出最终结果
    # print("\n=============== 最终审核结果 =============")
    # if not business_passed:
    #     print("审核结果: 规则审核未通过")
    #     print(f"失败原因: {business_reason}")
    # else:
    #     print(f"审核结果: {'通过' if passed else 'AI审核未通过'}")
    #     if not passed:
    #         print(f"失败原因: {', '.join(details.get('reasons', []))}")